In [0]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.applications.resnet50 import ResNet50
from keras.utils import np_utils
import matplotlib.pyplot as plt
%matplotlib inline

We will be using a set of "natural" images that can be found on Kaggle for our transfer learning. We will add 1 additional dense layer and an output layer to the ResNet50 model, these layers were stripped off when bringing the model in.

In [0]:
transfer_model = Sequential()
transfer_model.add(ResNet50(include_top=False,
                                     weights="imagenet",
                                     pooling="max"))
transfer_model.add(Dense(128,activation='relu'))
transfer_model.add(Dense(8,activation="softmax"))
transfer_model.layers[0].trainable = False

In [35]:
transfer_model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])
print(transfer_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_8 (Dense)              (None, 128)               262272    
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 1032      
Total params: 23,851,016
Trainable params: 263,304
Non-trainable params: 23,587,712
_________________________________________________________________
None


In [20]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Making use of the `ImageDataGenerator` in `keras.preprocessing` makes adding train and validation data easy

In [0]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [36]:
train_gen = datagen.flow_from_directory(directory="gdrive/My Drive/image_data/natural/natural_images",target_size=(224, 224), batch_size=10,subset="training",shuffle=True)
valid_gen = datagen.flow_from_directory(directory="gdrive/My Drive/image_data/natural/natural_images",target_size=(224, 224), batch_size=10,subset="validation")

Found 5522 images belonging to 8 classes.
Found 1377 images belonging to 8 classes.


In [37]:
transfer_model.fit_generator(train_gen, steps_per_epoch = 553, validation_data = valid_gen, validation_steps = 138, epochs = 5)
transfer_model.save('transfer.h5')

Epoch 1/5
553/553 [==============================] - 367s 663ms/step - loss: 8.1730 - acc: 0.4839 - val_loss: 10.0975 - val_acc: 0.1452
Epoch 2/5
553/553 [==============================] - 88s 160ms/step - loss: 7.3291 - acc: 0.5418 - val_loss: 11.6679 - val_acc: 0.1750
Epoch 3/5
553/553 [==============================] - 90s 163ms/step - loss: 7.3280 - acc: 0.5430 - val_loss: 12.6434 - val_acc: 0.1452
Epoch 4/5
553/553 [==============================] - 89s 161ms/step - loss: 7.2431 - acc: 0.5486 - val_loss: 11.9504 - val_acc: 0.1445
Epoch 5/5
553/553 [==============================] - 89s 161ms/step - loss: 7.2641 - acc: 0.5483 - val_loss: 13.1417 - val_acc: 0.1452


This model is far from perfect, but is good enough to illustrate the concept of "transfer learning"

In [38]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
  
model_file = drive.CreateFile({'title' : 'transfer.h5'})
model_file.SetContentFile('transfer.h5')
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})

     |████████████████████████████████| 993kB 3.4MB/s 


W0816 19:21:08.064916 140704377726848 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

GoogleDriveFile({'id': '1TU2WE5BrHbkYJx5u631oDZUaPPOFVTn2'})